# Ein einfaches Beispiel in Python

Alice möchte eine Nachricht an Bob verschicken. Dazu nutzt sie eine Funktion `send_message`, die eine Nachricht an den Empfänger übermittelt:

```python
send_message("Bob", "Hallo, Bob!")
// => True
```

Die Funktion gibt zurück, ob die Nachricht erfolgreich an den Empfänger übermittelt wurde.

In diesem Beispiel werden wir sehen, wie das Schichtenkonzept praktisch funktioniert. Wir nutzen aus der Anwendung heraus ein Protokoll der Anwendungsschicht, das wiederum Daten der Transportschicht mittels Sockets verschickt. Die Beispiele verzichten zur besseren Lesbarkeit auf jegliche Fehlerbehandlung.


## 1. Alices Implementierung von `send_message`

Als erstes müssen Alice und Bob sich auf ein *Protokoll* der Anwendungsschicht einigen, über das sie die Nachricht übertragen können. Dieses Protokoll müssen dann beide implementieren.

### HTTP

Hier benutzen sie das [Hypertext Transfer Protocol (HTTP)](https://de.wikipedia.org/wiki/Hypertext_Transfer_Protocol), mit dem üblicherweise Webseiten übertragen werden. Es wird aber auch für viele andere Dienste genutzt, da es vielseitig und sehr einfach ist. Es gibt auch Protokolle, die besser für den Austausch von Nachrichten geeignet sind, jedoch eignen sie sich nicht für ein so einfaches Beispiel.

Alice schreibt also die Funktion `send_message` und nutzt HTTP zum verschicken der Nachricht.

In [73]:
def send_message(to, message):
    # Wie genau wir den Host aus dem "Benutzernamen" (to) ermitteln, hängt von unserer Applikation ab.
    # Der Einfachheit halber legen wir 'www.nm.ifi.lmu.de' fest, aber
    # das könnte z.B. auch in einer Datenbank gespeichert sein.
    host = "www.nm.ifi.lmu.de"
    
    # Verschicken der Nachricht über HTTP POST
    status = post_http(host, "/teaching/Vorlesungen/2024ss/rn/message/", message)
    
    # Gebe True zurück, wenn der Response-Status 200 ist
    print("Message sent to", to, "with status", status)
    return status == 200

HTTP folgt der Client-Server-Architektur. Die PDU in HTTP nennt sich *Nachricht*, wird durch Absetzen einer HTTP-Anfrage (Request) durch den Client initialisiert. Dieser Request enthält neben unserer Nachricht als Nutzlast bzw. Nutzdaten einen HTTP-Header oder kurz Header, der Steuerinformationen enthält. Der HTTP-Server quittiert den Request mit einer Antwort (Response), die u.a. anzeigt, ob der Request erfolgreich empfangen wurde.

Ein HTTP-Request hat die Form: `[METHODE] [PFAD] HTTP/1.1\r\n[FELDER]\r\n\r\n[NUTZLAST]`, also z.B.

```
POST /messages HTTP/1.1
Host: bob.example.com
Content-Length: 11

Hallo, Bob!
```

Die Zeichenfolge `\r\n` steht für eine neue Zeile. Alice möchte aus der Nachricht (Nutzdaten) und dem Empfänger (Steuerdaten) einen HTTP-Request erzeugen, diesen übertragen und die Antwort des HTTP-Server erhalten. Dazu implementiert sie eine Funktion `post_http(target, body)`.

In [74]:
def post_http(host, path, payload):
    # Erzeugen eines Requests
    request = create_http_request("POST", path, host, payload)
    
    # Verschicken des Requests über TCP
    http_port = 443
    response = transmit_with_tcp(host, http_port, request)
    
    # Interpretieren der erhaltenen Antwort als String
    response_string = response.decode('utf-8')

    # Extrahieren des "Status" der Antwort via Regex
    # Dieser extrahiert aus "HTTP/1.1 xxx The Status Message" die xxx
    match = re.search(r"^HTTP/1\.1 (\d{3}) .+", response_string)
    status = int(match.group(1))

    # Antwort zurückgeben
    return status

Die Funktion erzeugt also aus aus einer Nachricht und einigen Steuerdaten einen Request, verschickt diesen und gibt den *status* der Antwort zurück. Zuerst muss Alice die Funktion schreiben, die den Request erstellt:

In [75]:
def create_http_request(method, path, host, body = ""):
    content_length = len(body)
    header = "%s %s HTTP/1.1\r\nHost: %s\r\nContent-Length: %s" %(method, path, host, content_length)
    request = "%s\r\n\r\n%s" % (header, body)
    return request.encode('utf-8')

Diese Funktion gibt die Zeichenkette als UTF-8 enkodierte Binärdaten zurück. Der Request ist - wie oben zu sehen - einfach nur Text, jedoch können wir auf unterliegenden Schichten nur Bytes also Binärdaten übertragen. Man kann die Bytes wieder dekodieren (`decode`) und die Zeichenkette ausgeben:

In [76]:
print(create_http_request("GET", "/", "www.nm.ifi.lmu.de").decode('utf-8'))

GET / HTTP/1.1
Host: www.nm.ifi.lmu.de
Content-Length: 0




Alice nutzt die Methode `POST` und den Pfad `/teaching/Vorlesungen/2024s/rn/message/`. Über die genaue Semantik dieser beiden Parameter des HTTP-Requests muss zwischen Alice und Bob Einigkeit bestehen. Vom Standard werden nur einige Vorgaben gemacht, z.B. dass ein `GET`-Request die mit dem Pfad beschriebenen Resource in der Antwort zurückliefert.

Die Funktion `transmit_with_tcp` soll nun den Request übertragen. Dafür ist die Transportschicht zuständig. Jemand, der die Funktion `post_http` aufruft, muss nichts über das gewählte Protokoll dieser Schicht wissen. Hier wird das **Schichtenprinzip** deutlich: wie `transmit_with_tcp` die Übertragung gewährleistet, ist für den Aufrufer der Funktion **gänzlich transparent**.

Für HTTP ist als Transportschichtprotokoll TCP im RFC festgeschrieben. `transmit_with_tcp` nimmt den HTTP-Request als Nutzdaten entgegen und überträgt ihn via TCP.

### TCP Sockets

Zum zuverlässigen übertragen von Datenströmen wurde TCP entworfen. Da TCP *verbindungsorientiert* ist, muss erst eine Verbindung aufgebaut werden, bevor Datenströme verschickt bzw. empfangen werden können.

Ein TCP-Socket besteht aus einem paar von *IP-Adresse* und einem *Port*. Die IP-Adresse beschreibt dabei den Host und der Port dient zum Multiplexing der Applikationen auf dem Host. Viele Protokolle haben typische Ports zugewiesen, unter denen die Server erreichbar sind, damit nicht explizit ein Port zwischen Server und Client vereinbart werden muss.

Alice ist in diesem Fall der Client, da sie eine Verbindung zu Bob bzw. seinem Server-Prozess aufbaut. Als Client hat sie vier Schritte zu vollziehen:

1. Verbindung aufbauen (connect)
2. Daten senden (send)
3. Daten empfangen (recv)
4. Verbindung schließen (close)

All das passiert in `transmit_with_tcp`:

In [77]:
import socket
import re
import ssl

context = ssl.create_default_context()

def transmit_with_tcp(host, port, data):
    # Erzeugt in "s" einen TCP-Socket (SOCK_STREAM) über IPv4 (AF_INET)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        with context.wrap_socket(s, server_hostname=host) as ss:
            # 1. Verbindung zum Server-Socket (host, port) wird aufgebaut
            ss.connect((host, port))

            # 2. Die Daten werden verschickt
            ss.sendall(data)

            # 3. Die Antwort wird empfangen (bis zu 4096 Bytes)
            response_bytes = ss.recv(4096)      

            # 4. Verbindung schließen (geschieht automatisch beim Verlassen der with-Umgebung)
            return response_bytes

Hier kann man die vier Phasen der Verbindung gut erkennen. Wollte Alice viele Nachrichten hintereinander verschicken, würde sie die Verbindung länger offen halten und `send[all]` und `recv` mehrfach auf dem einmal geöffneten Socket `s` aufrufen. `s.close()` würde sie in dem Fall natürlich erst nach dem Versenden aller Nachrichten aufrufen.

### Verschicken der Nachricht

Nun kann Alice `send_message` aus ihrer Applikation heraus aufrufen. Aus Sicht der Anwendung sind Aufgrund der Trennung der Schichten die dabei benutzten Protokolle transparent.

In [78]:
if send_message("Bob", "Hallo, Bob!") == True:
    print("Nachricht erfolgreich verschickt.")
else:
    print("Verschicken fehlgeschlagen.")    

SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)

Alice hat die Nachricht erfolgreich übertragen und dafür eine Bestätigung erhalten.

## 2. Bobs Implementierung von `receive_message`

Wegen der Client-Server-Architektur muss Bob einen Server betreiben und auf Nachrichten warten. Er muss also die eine Funktion `receive_message` bereitstellen, die genau dies tut.

In [ ]:
def receive_message():
    payload = handle_http_request()
    return payload

Das ist einfach: `handle_http_request` blockiert so lange, bis ein HTTP-Request von einem Client eingegangen ist, extrahiert die Nutzlast (`payload`) und gibt eine Antwort zurück.

### TCP Server

Bob muss nun das passende Gegenstück zu Alices Client-Implementierung bereitstellen. In `handle_http_request` sieht man jetzt wieder den Übergang von Anwendungs- zu Transportschicht.

Bob muss jetzt einen Socket erstellen, auf dem er auf eingehende Verbindungen von Clients lauscht, diese entgegennimmt und ggf. antwortet. Das passiert ebenfalls in mehreren Schritten:

1. Socket an ein Host/Port-Tupel *binden* (`bind`)
2. Auf dem Socket auf einkommende Verbindungen *lauschen*
3. Einkommende Verbindungen annehmen, was einen neuen Socket für den Verbundnen Client erzeugt (`accept`)
4. Socket Schließen (`close`)

In [ ]:
import socket

def handle_http_request():
    # HTTP benutzt TCP
    host = b'' # Lauschen auf allen möglichen Adressen, die auf diesem Host zur Verfügung stehen
    port = 8080 # Muss mit Client übereinstimmen
    
    # Erzeugen eines Puffers zum Empfangen eines Requests
    request_bytes = b''
    
    # Erzeugen eines Sockets analog zum Client
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        # Binden des Sockets an Port "port"
        s.bind((host, port))
        
        # Lauschen auf eine Verbindung
        s.listen(1)
        
        # Akzeptieren der Verbindung, erzeug einen neuen Socket "conn"
        conn, addr = s.accept()

        with conn:
            # Empfangen des HTTP-Requests von bis zu 4096 Bytes
            request_bytes = conn.recv(4096)

            # Request erhalten: antworten mit einer einfachen HTTP-Response
            http_response = b'HTTP/1.1 200 OK\r\nConnection: close\r\n'
            conn.sendall(http_response)
        
    # Erzeugen einer Zeichenkette aus den Binärdaten
    request = request_bytes.decode('utf-8')
    
    # Payload auslesen: zur Erinnerung ein Request hat die Form 'HEADER\r\n\r\nPAYLOAD'
    # Man kann also alles nach \r\n\r\n als Payload ansehen
    payload = request.split("\r\n\r\n")[1]
    
    return payload

`handle_http_request` öffnet also einen Socket, lauscht auf eingehende Verbindungen, nimmt einen Bytestrom entgegen, interpretiert ihn als HTTP-Request und Antwortet mit einer einfachen `200 OK` Response. Außerdem extrahiert es noch den Payload (Alices Nachricht), die im Request mitgeschickt wurde.

Hier sei noch einmal darauf hingewiesen, dass diese Implementierung einige Vereinfachungen macht. Zum Beispiel nimmt sie immer nur eine Anfrage entgegen und die Anfrage darf nicht größer als 4096 Bytes sein. In der Praxis arbeitet man deshalb mit while-Schleifen, die so lange Inhalt lesen, bis der Request komplett empfangen wurde. Dafür nutzt man das `Content-Length`-Headerfeld, um so lange Daten in `request_bytes` zu lesen, bis alle Bytes empfangen wurden.

### Testen des Servers

Wenn man dieses Notebook auf dem eigenen Rechner ausführt, kann man die folgende Zeile einkommentieren (das `#` entfernen) und ausführen. Dann wird ein HTTP-Server gestartet, der genau eine Nachricht entgegen nimmt.

In [ ]:
# receive_message()

Ist der Server gestartet (links erscheint `In [*]`), so kann man per

```
curl -4 -v -d 'Hallo, Bob!' localhost:8080
```

In einer Shell die Nachricht verschicken, sie wird dann hier angezeigt. Alternativ kann man das Notebook ein zweites Mal starten und den `host` im Client auf `localhost` anpassen.

## Fazit

An diesem Beispiel kann man das Schichtenkonzept in der Praxis sehen.

- Normalerweise schreibt man einen HTTP-Client/Server nicht selbst, sondern nutzt fertige Bibliotheken. Damit reduziert sich auch die Komplexität deutlich: man ruft das Equivalent zu `post_http` in der Bibliothek auf und muss sich um den Rest nicht kümmern.
- Die Applikation nutzt Protokolle der Applikationsschicht (HTTP in diesem Fall), die wiederum Protokolle der Transportschicht nutzen (TCP). Das geht natürlich weiter bis auf die Bitübertragungsschicht herunter, was den [Rahmen eines einfachen Beispiels](https://tools.ietf.org/html/rfc1180) sprengen würde.
- Für den Anwendungsprogrammierer sind die Details der unteren Schichten transparent. Würde `send_message` statt HTTP ein komplett anderes Protokoll nutzen, müsste sich der Funktionsaufruf nicht ändern. Gleiches gilt auch jeweils für die tieferen Schichten!